In [8]:
#실시간 4초 촬영 후 촬영 창 닫고 x,y,z 좌표를 csv파일로 저장하는 코드입니다.

import cv2
import mediapipe as mp
import pandas as pd
import time

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# 빈 dataframe 생성
df = pd.DataFrame()

# webcam / video 불러오기
cap = cv2.VideoCapture(0)
start_time = time.time()  # 현재 시간을 기록

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      break

    # 3초 경과 시 루프 종료
    if time.time() - start_time > 4:
        break

    # 이미지 처리
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # pose data 저장
    x = []
    for k in range(33):
        x.append(results.pose_landmarks.landmark[k].x)
        x.append(results.pose_landmarks.landmark[k].y)
        x.append(results.pose_landmarks.landmark[k].z)
        x.append(results.pose_landmarks.landmark[k].visibility)

    # list x를 dataframe으로 변경
    tmp = pd.DataFrame(x).T

    # dataframe에 정보 쌓아주기
    df = pd.concat([df, tmp])

    # 이미지에 포즈 그리기
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    # 이미지 표시
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()  # 모든 창 닫기

# 결과를 'hello.csv' 파일로 저장
df.to_csv('data_data.csv')

